## Effect of varying enzyme affinities

### Vary Km ClO4- of PRB

Vary the substrate affinity of the perchlorate reducing bacterium for perchlorate

### Vary Km ClO4- and Km ClO3-

- Compare to final CRB/PRB ratio

In [ ]:
def vary_PRB_Kms(z_variable, filename):
    
    import matplotlib.colors as colors

    # Iterate through model
    
    # Initial concentrations (M)
    x0 = [0.0001, # PRB 
          0.0001, # CRB 
          0.0125, # Acetate
          0.01, # Perchlorate
          0.0, # Chlorate
          0.0, # Cum. clorate to PRB
          0.0] # Cum. clorate to CRB
    
    # Initialize X-, Y-, and Z- dimensions
    X = np.array(x_range)
    Y = np.array(x_range)
    Z = np.array(x_range)
    
    for y in y_range:
    
        output = []
        
        # Plot by varying concentration
        for x in x_range:
            
            # Replace Km values
            ks_clo4[id_prb] = x
            ks_clo3[id_prb] = y
            
            # Calculate from each initial condition
            state = odeint(eca_kinetics, x0, t_span)
            df = odeint_to_dataframe(state)
            statistics = dataframe_statistics(df)

            output.append(statistics[z_variable])
        
        x_row = x_range
        X = np.vstack((X,x_row))
            
        y_row = np.array([y]*len(x_range))
        Y = np.vstack((Y,y_row))
            
        z_row = output
        Z = np.vstack((Z,z_row))
    
    # Plot as heatmap with pcolormesh

    X = X[1:] # X-dimension
    Y = Y[1:] # Y-dimension
    Z = Z[1:-1, :-1] # Z-dimension within X-Y bounds

    # Normalization
    cmap = plt.get_cmap(color_map)
    levels = MaxNLocator(nbins=1000).tick_values(Z.min(), Z.max())
    
    if z_scale == 'log':
        # https://matplotlib.org/users/colormapnorms.html
        if z_variable == 'f:CRB/PRB':
            z_bound = np.max([1/Z.min(),Z.max()])
            norm = colors.LogNorm(vmin=1/z_bound, vmax=z_bound)
        else:
            norm = colors.LogNorm(vmin=Z.min(), vmax=Z.max())
    else:
        # https://matplotlib.org/api/_as_gen/matplotlib.colors.BoundaryNorm.html
        norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True,)

    
    fig, ax = plt.subplots(figsize=(8,6))
    im = ax.pcolormesh(X, Y, Z, cmap=cmap, norm=norm)
    fig.colorbar(im, ax=ax)
    
    ax.set_xscale(x_scale, basex=2)
    ax.set_yscale(y_scale, basey=2)
    ax.set_title(z_variable)
    ax.set_xlabel("PRB Km ClO4- (M)")
    ax.set_ylabel("PRB Km ClO3- (M)")
    
    plt.tight_layout()
    plt.savefig(filename)
    
    return plt.show()

# Reset values
ks_clo4, ks_clo3, ks_acet, ki_clo4, ki_clo3, ki_acet, mu, m = reset_parameters()
x0 = reset_default_concentrations()

# Z: Final ratio
z_variable = 'f:CRB/PRB'
z_scale = 'log'
color_map = 'RdBu'

# Y: Km ClO3- PRB
y_base2_min  = -26
y_base2_max  = -2
y_base2_steps = 1 + y_base2_max - y_base2_min
y_range = np.logspace(start=y_base2_min,stop=y_base2_max,base=2,num=y_base2_steps)
y_scale = 'log'

# X: Km ClO4- PRB
x_base2_min  = -26
x_base2_max  = -2
x_base2_steps = 1 + x_base2_max - x_base2_min
x_range = np.logspace(start=x_base2_min,stop=x_base2_max,base=2,num=x_base2_steps)
x_scale = 'log'

print ("Evolutionary landscape of Pcr evolution")
vary_PRB_Kms(z_variable,"vary-km.svg")

# Need to adjust cmap to match some absolute CRB/PRB endpoints